# Ant Colony Optimization

Cada hormiga representa una solución al problema.
Entorno: grafo del problema del viajante simétrico a resolver.
Se representa como una matriz de adyacencia en la que el valor de la posición [i][j] es la distancia entre las ciudades i y j.

In [ ]:
import random
import copy
import json
import time
import tsplib95

### Función cargaCasoDePrueba

Lee un caso de prueba del fichero con nombre _nombreFichero_ y devuelve la matriz de adyacencia correspondiente.

In [ ]:
def cargaCasoDePrueba(nombreFichero):
    with open(nombreFichero, 'r') as archivo:
        matrizAdy = json.loads(archivo.read())
    return matrizAdy

### Función tsplibAMatrizAdyacencia

Convierte el problema dado del formato tsplib a la matriz de adyacencia correspondiente.

In [ ]:
def tsplibAMatrizAdyacencia(problema):
    matrizAdy = [[]] * problema.dimension
    for i in range(problema.dimension):
        matrizAdy[i] = [problema.get_weight(i+1,j+1) for j in range(problema.dimension)]
    return matrizAdy

### Función cargaCasoTsplib

Carga un problema de la tsplib y lo devuelve ya en forma de matriz de adyacencia.

In [ ]:
def cargaCasoTsplib(nombreFichero):
    problema = tsplib95.load(nombreFichero)
    return tsplibAMatrizAdyacencia(problema)

### Función guardarResultados

Dadas las ejecuciones de un algoritmo, las guarda en un fichero formateado adecuadamente para posteriormente cargarlas en el notebook Graficas.ipynb

In [ ]:
def guardarResultados(ejecuciones, problema, algoritmo, num):
    nombreFichero = problema + "_" + algoritmo + "_" + str(num) + ".txt"
    with open(nombreFichero, 'a') as archivo:
        for ejecucion in ejecuciones:
            archivo.write(str(ejecucion[0]) + '\n')
            for hormiga in ejecucion[1]:
                archivo.write(str(hormiga))
            archivo.write('FIN\n')

## Arista
La clase Arista nos permite representar un camino entre dos ciudades concretas. Ayuda a guardar el valor de feromona que hay en cada momento en el camino.

In [ ]:
class Arista:
    def __init__(self, desde, hasta, distancia=None, feromonaInicial=None):
        self.desde = desde
        self.hasta = hasta
        self.distancia = 1 if distancia is None else distancia
        self.feromonaInicial = 0.1 if feromonaInicial is None else feromonaInicial
        self.feromona = feromonaInicial

## Entorno
La clase Entorno representa una instancia del problema del viajante a resolver. Contiene una serie determinada de Aristas que crean el grafo a resolver.

In [ ]:
class Entorno:
    def __init__(self, matrizAdy, feromonaInicial=None):
        self.conjuntoAristas = set()
        self.aristas = self.creaAristas(matrizAdy, feromonaInicial)
        self.numVertices = len(matrizAdy)
        
    def creaAristas(self, matrizAdy, feromonaInicial=None):
        aristas = {}
        for i in range(len(matrizAdy)):
            for j in range(i):
                # En MMAS el valor inicial de la feromona es siempre el máximo permitido.
                arista = Arista(i, j, matrizAdy[i][j],
                                feromonaInicial = feromonaInicial)
                aristas[i,j] = arista
                aristas[j,i] = arista
                self.conjuntoAristas.add(arista)
        return aristas
    
    def resetearFeromonas(self):
        for arista in self.conjuntoAristas:
            arista.feromona = arista.feromonaInicial

## Hormiga
La clase Hormiga representa una solución concreta al problema planteado en el entorno.
Realiza un trayecto concreto entre las ciudades eligiendo sus movimientos en base a las feromonas de las aristas. Posee algunos métodos distintos según la versión de ACO que se esté usando.

In [ ]:
class Hormiga:
    # Alfa: importancia dada a las feromonas
    # Beta: importancia dada a la distancia
    def __init__(self):
        self.nodoActual = None
        self.inicio = None
        self.entorno = None
        self.visitados = []
        self.porVisitar = []
        self.trayecto = []
        self.distancia = 0
        
    def __str__(self):
        resultado = str(self.distancia) + '\n' + str(self.visitados) + '\n'
        return resultado
    
    def __repr__(self):
        resultado = str(self.distancia) + '\n' + str(self.visitados) + '\n'
        return resultado
        
    # Nos va a interesar poder ordenar las hormigas según la distancia de su solución
    # para poder ordenarlas y quedarnos con la mejor.
    def __eq__(self,other):
        return self.distancia == other.distancia
    
    def __lt__(self,other):
        return self.distancia < other.distancia
        
    def inicializar(self, entorno, inicio):
        self.entorno = entorno
        self.inicio = inicio
        self.nodoActual = inicio
        self.visitados = [inicio]
        self.porVisitar = [v for v in range(entorno.numVertices) if v != inicio]
        self.trayecto = []
        self.distancia = 0
        return self
    
    # Elige un movimiento según Ant System y lo efectúa devolviendo la arista elegida.
    def movimientoAS(self):
        eleccion = self.elegirMovimientoAS()
        return self.realizaMovimiento(eleccion)
    
    # Elige el movimiento según Ant System. Devuelve la arista a usar.
    def elegirMovimientoAS(self):
        # Sólo queda volver al nodo original
        if not self.porVisitar:
            return self.entorno.aristas[self.nodoActual, self.inicio]
        # Sólo queda un nodo por visitar
        if len(self.porVisitar) == 1:
            return self.entorno.aristas[self.nodoActual, self.porVisitar[0]]
        listaAristas = []
        for nodo in self.porVisitar:
            listaAristas.append(self.entorno.aristas[self.nodoActual, nodo])
        listaPesos = [self.calculaPesoAS(arista) for arista in listaAristas]
        pesoAcumulado = sum(listaPesos)
        if(pesoAcumulado == 0):
            return listaAristas[0]
        probabilidades = []
        for peso in listaPesos:
            # Calculamos las probabilidades en %
            probabilidades.append(100 * peso/pesoAcumulado)
        return random.choices(listaAristas, weights=probabilidades)[0]
    
    # Elige un movimiento según Ant Colony System y lo efectúa devolviendo la arista elegida.
    def movimientoACS(self):
        eleccion = self.elegirMovimientoACS()
        return self.realizaMovimiento(eleccion)
    
    # Elige el movimiento según Ant Colony System. Devuelve la arista a usar.
    def elegirMovimientoACS(self):
        # Sólo queda volver al nodo original
        if not self.porVisitar:
            return self.entorno.aristas[self.nodoActual, self.inicio]
        # Sólo queda un nodo por visitar
        if len(self.porVisitar) == 1:
            return self.entorno.aristas[self.nodoActual, self.porVisitar[0]]
        # q número sacado de una distribución uniforme entre 0 y 1
        q = random.uniform()
        if q <= q0:
            # Elegimos según ACS
            listaAristas = []
            for nodo in self.porVisitar:
                listaAristas.append(self.entorno.aristas[self.nodoActual, nodo])
            aristaMax = max([calculapesoACS(arista) for arista in listaAristas])
            return aristaMax
        else:
            # Elegimos igual que en Ant System
            return elegirMovimientoAS(self)
        
    # A mayor peso, mayor probabilidad de elección de la arista.
    def calculaPesoAS(self, arista):
        tau = arista.feromona
        eta = 1/arista.distancia
        return (tau ** alfa) * (eta ** beta)
    
    def calculaPesoACS(self, arista):
        tau = arista.feromona
        eta = 1/arista.distancia
        return tau * (eta ** beta)
    
    def realizaMovimiento(self, arista):
        hasta = arista.hasta if self.nodoActual == arista.desde else arista.desde
        if  not (hasta == self.inicio):
            self.visitados.append(hasta)
            self.porVisitar.remove(hasta)
        self.distancia += arista.distancia
        self.nodoActual = hasta
        return arista
        

## Funciones

### Función crearColonia

Esta función genera una nueva colonia de hormigas y las inicializa en posiciones aleatorias del grafo. Requiere que hayamos definido previamente las variables _alfa_ y _beta_ en el notebook.

In [ ]:
def crearColonia(entorno, numHormigas):
    colonia = []
    for i in range(numHormigas):
            nuevaHormiga = Hormiga().inicializar(entorno, random.randrange(entorno.numVertices))
            colonia.append(nuevaHormiga)
    return colonia

### Función reiniciarColonia

Inicializa todas las hormigas de la colonia y les asigna un nuevo punto de inicio aleatorio.

In [ ]:
def reiniciarColonia(colonia):
    for hormiga in colonia:
        hormiga.inicializar(hormiga.entorno, random.randrange(hormiga.entorno.numVertices))

### Función encontrarSolucionesAS

Realiza todos los movimientos de todas las hormigas según lo indicado en Ant System para que cada hormiga de toda la colonia tenga una solución completa.

In [ ]:
# Dos bucles: ya que todas las hormigas tienen que moverse por los nodos hasta haberlos visitado todos.
def encontrarSolucionesAS(entorno, colonia):
    for i in range(entorno.numVertices):
        for hormiga in colonia:
            arista = hormiga.movimientoAS()
            hormiga.trayecto.append(arista)
    colonia = sorted(colonia)
    return colonia[0]

### Función actualizarAristaLocal

Utilizada en Ant Colony System para decaer la feromona de la arista tras cada paso.

In [ ]:
def actualizarAristaLocal(arista):
    arista.feromona = (1-phi) * arista.feromona + phi * arista.feromonaInicial

### Función encontrarSolucionesACS

Realiza todos los movimientos de todas las hormigas según lo indicado en Ant Colony System para que cada hormiga de toda la colonia tenga una solución completa. Realiza la actualización local de las aristas, que hace que decaiga la feromona de cada arista tras cada paso de construcción.

In [ ]:
# Dos bucles: ya que todas las hormigas tienen que moverse por los nodos hasta haberlos visitado todos.
def encontrarSolucionesACS(entorno, colonia):
    for i in range(entorno.numVertices):
        for hormiga in colonia:
            arista = hormiga.movimientoAS()
            hormiga.trayecto.append(arista)
            actualizarAristaLocal(arista)
    colonia = sorted(colonia)
    return colonia[0]

### Función actualizarGlobalAS

Actualiza todas las aristas del entorno según lo indicado en AntSystem; decayendo su feromona según el valor rho predefinido en el notebook y aumentándola según las hormigas que hayan pasado por cada arista y la longitud final del camino de dicha hormiga.

In [ ]:
def actualizarGlobalAS(entorno, colonia):
    # Decaer feromona
    for arista in entorno.conjuntoAristas:
        arista.feromona = (1 - rho) * arista.feromona
    # Aumentar feromona según uso por parte de las hormigas.
    # En AS la fitness function es 1/distancia de la solución
    for hormiga in colonia:
        for arista in hormiga.trayecto:
            arista.feromona += 1/hormiga.distancia

### Función actualizarMejorACS

Decae todas las aristas pero actualiza sólo las aristas de la mejor solución encontrada, según se requiere en Ant Colony System.

In [ ]:
def actualizarMejorACS(hormiga):
    # Decaer feromona
    for arista in entorno.conjuntoAristas:
        arista.feromona = (1 - rho) * arista.feromona
    # Aumentar feromona de las aristas de la mejor solución
    for arista in hormiga.trayecto:
        arista.feromona += rho * 1/hormiga.distancia

### Función actualizarMejorMMAS

En MAX-MIN Ant System de nuevo sólo actualiza la mejor hormiga, pero lo hace de manera distinta a ACS, por eso necesitamos una función nueva. Además nos aseguramos de mantener las feromonas entre los límites permitidos.

In [ ]:
def actualizarMejorMMAS(hormiga):
    # Decaer feromona
    for arista in entorno.conjuntoAristas:
        arista.feromona = (1 - rho) * arista.feromona
        arista.feromona = max(arista.feromona, minFeromona)
    # Aumentar feromona de las aristas de la mejor solución
    for arista in hormiga.trayecto:
        arista.feromona += 1/hormiga.distancia
        arista.feromona = max(arista.feromona, minFeromona)
        arista.feromona = min(arista.feromona, maxFeromona)

### Funciones AntSystem

Dado un entorno, un número de iteraciones, y un número de hormigas, aplica la metaheurística del Ant System para intentar obtener una buena solución al problema del TSP. Devuelve la mejor solución encontrada tras todas las iteraciones. Cada una usa una condición de parada diferente.

In [ ]:
def AntSystemIteraciones(entorno, numIteraciones, numHormigas):
    entorno.resetearFeromonas()
    mejorSolucionGlobal = None
    colonia = crearColonia(entorno, numHormigas)
    for i in range(numIteraciones):
        reiniciarColonia(colonia)
        mejorSolucionActual = encontrarSolucionesAS(entorno, colonia)
        if mejorSolucionGlobal is None or mejorSolucionActual < mejorSolucionGlobal:
            mejorSolucionGlobal = copy.deepcopy(mejorSolucionActual)
        actualizarGlobalAS(entorno, colonia)
    return mejorSolucionGlobal

In [ ]:
def AntSystemTiempo(entorno, numHormigas):
    tiempoInicio = time.time()
    listaSoluciones = []
    entorno.resetearFeromonas()
    mejorSolucionGlobal = None
    colonia = crearColonia(entorno, numHormigas)
    # Paramos el algoritmo tras 300 segundos (5 minutos)
    while time.time() - tiempoInicio < 300:
        reiniciarColonia(colonia)
        mejorSolucionActual = encontrarSolucionesAS(entorno, colonia)
        if mejorSolucionGlobal is None or mejorSolucionActual < mejorSolucionGlobal:
            mejorSolucionGlobal = copy.deepcopy(mejorSolucionActual)
        listaSoluciones.append(mejorSolucionGlobal)
        actualizarGlobalAS(entorno, colonia)
    tiempoFin = time.time()
    tiempo = tiempoFin - tiempoInicio
    return listaSoluciones, tiempo

In [ ]:
def AntSystem(entorno, numHormigas):
    tiempoInicio = time.time()
    # Si no se mejora la solución después de maxSeguidas iteraciones, se para el algoritmo.
    seguidas = 0
    maxSeguidas = 10000
    listaSoluciones = []
    entorno.resetearFeromonas()
    mejorSolucionGlobal = None
    colonia = crearColonia(entorno, numHormigas)
    while seguidas < maxSeguidas:
        reiniciarColonia(colonia)
        mejorSolucionActual = encontrarSolucionesAS(entorno, colonia)
        if mejorSolucionGlobal is None or mejorSolucionActual < mejorSolucionGlobal:
            seguidas = 0
            mejorSolucionGlobal = copy.deepcopy(mejorSolucionActual)
        else:
            seguidas += 1
        listaSoluciones.append(mejorSolucionGlobal)
        actualizarGlobalAS(entorno, colonia)
    tiempoFin = time.time()
    tiempo = tiempoFin - tiempoInicio
    return listaSoluciones, tiempo

### Funciones AntColonySystem

Dado un entorno, un número de iteraciones, y un número de hormigas, aplica la metaheurística del Ant Colony System para intentar obtener una buena solución al problema del TSP. Devuelve la mejor solución encontrada tras todas las iteraciones. Cada una usa una condición de parada diferente.

In [ ]:
def AntColonySystemIteraciones(entorno, numIteraciones, numHormigas):
    entorno.resetearFeromonas()
    mejorSolucionGlobal = None
    colonia = crearColonia(entorno, numHormigas)
    for i in range(numIteraciones):
        reiniciarColonia(colonia)
        mejorSolucionActual = encontrarSolucionesACS(entorno, colonia)
        if mejorSolucionGlobal is None or mejorSolucionActual < mejorSolucionGlobal:
            mejorSolucionGlobal = copy.deepcopy(mejorSolucionActual)
        actualizarMejorACS(mejorSolucionGlobal)
    return mejorSolucionGlobal

In [ ]:
def AntColonySystemTiempo(entorno, numHormigas):
    tiempoInicio = time.time()
    listaSoluciones = []
    entorno.resetearFeromonas()
    mejorSolucionGlobal = None
    colonia = crearColonia(entorno, numHormigas)
    # Paramos el algoritmo tras 300 segundos (5 minutos)
    while time.time() - tiempoInicio < 300:
        reiniciarColonia(colonia)
        mejorSolucionActual = encontrarSolucionesACS(entorno, colonia)
        if mejorSolucionGlobal is None or mejorSolucionActual < mejorSolucionGlobal:
            mejorSolucionGlobal = copy.deepcopy(mejorSolucionActual)
        listaSoluciones.append(mejorSolucionGlobal)
        actualizarMejorACS(mejorSolucionGlobal)
    tiempoFin = time.time()
    tiempo = tiempoFin - tiempoInicio
    return listaSoluciones, tiempo

In [ ]:
def AntColonySystem(entorno, numHormigas):
    tiempoInicio = time.time()
    # Si no se mejora la solución después de maxSeguidas iteraciones, se para el algoritmo.
    seguidas = 0
    maxSeguidas = 10000
    listaSoluciones = []
    entorno.resetearFeromonas()
    mejorSolucionGlobal = None
    colonia = crearColonia(entorno, numHormigas)
    while seguidas < maxSeguidas:
        reiniciarColonia(colonia)
        mejorSolucionActual = encontrarSolucionesACS(entorno, colonia)
        if mejorSolucionGlobal is None or mejorSolucionActual < mejorSolucionGlobal:
            seguidas = 0
            mejorSolucionGlobal = copy.deepcopy(mejorSolucionActual)
        else:
            seguidas += 1
        listaSoluciones.append(mejorSolucionGlobal)
        actualizarMejorACS(mejorSolucionGlobal)
    tiempoFin = time.time()
    tiempo = tiempoFin - tiempoInicio
    return listaSoluciones, tiempo

### Funciones MaxMinAntSystem

Dado un entorno, un número de iteraciones, y un número de hormigas, aplica la metaheurística del MAX-MIN Ant System para intentar obtener una buena solución al problema del TSP. Devuelve la mejor solución encontrada tras todas las iteraciones. Cada una usa una condición de parada diferente.

In [ ]:
def MaxMinAntSystemIteraciones(entorno, numIteraciones, numHormigas):
    entorno.resetearFeromonas()
    mejorSolucionGlobal = None
    colonia = crearColonia(entorno, numHormigas)
    for i in range(numIteraciones):
        reiniciarColonia(colonia)
        mejorSolucionActual = encontrarSolucionesAS(entorno, colonia)
        if mejorSolucionGlobal is None or mejorSolucionActual < mejorSolucionGlobal:
            mejorSolucionGlobal = copy.deepcopy(mejorSolucionActual)
        actualizarMejorMMAS(mejorSolucionGlobal)
    return mejorSolucionGlobal

In [ ]:
def MaxMinAntSystemTiempo(entorno, numHormigas):
    tiempoInicio = time.time()
    listaSoluciones = []
    entorno.resetearFeromonas()
    mejorSolucionGlobal = None
    colonia = crearColonia(entorno, numHormigas)
    # Paramos el algoritmo tras 300 segundos (5 minutos)
    while time.time() - tiempoInicio < 300:
        reiniciarColonia(colonia)
        mejorSolucionActual = encontrarSolucionesAS(entorno, colonia)
        if mejorSolucionGlobal is None or mejorSolucionActual < mejorSolucionGlobal:
            mejorSolucionGlobal = copy.deepcopy(mejorSolucionActual)
        listaSoluciones.append(mejorSolucionGlobal)
        actualizarMejorMMAS(mejorSolucionGlobal)
    tiempoFin = time.time()
    tiempo = tiempoFin - tiempoInicio
    return listaSoluciones, tiempo

In [ ]:
def MaxMinAntSystem(entorno, numHormigas):
    tiempoInicio = time.time()
    # Si no se mejora la solución después de maxSeguidas iteraciones, se para el algoritmo.
    seguidas = 0
    maxSeguidas = 10000
    listaSoluciones = []
    entorno.resetearFeromonas()
    mejorSolucionGlobal = None
    colonia = crearColonia(entorno, numHormigas)
    while seguidas < maxSeguidas:
        reiniciarColonia(colonia)
        mejorSolucionActual = encontrarSolucionesAS(entorno, colonia)
        if mejorSolucionGlobal is None or mejorSolucionActual < mejorSolucionGlobal:
            seguidas = 0
            mejorSolucionGlobal = copy.deepcopy(mejorSolucionActual)
        else:
            seguidas += 1
        listaSoluciones.append(mejorSolucionGlobal)
        actualizarMejorMMAS(mejorSolucionGlobal)
    tiempoFin = time.time()
    tiempo = tiempoFin - tiempoInicio
    return listaSoluciones, tiempo

## Pruebas

#### Definiciones de cada parámetro:

alfa = importancia dada a las feromonas

beta = importancia dada a la distancia

rho = tasa de evaporación

q0 = parámetro límite de la variable q en Ant Colony System

phi = coeficiente de decadencia de feromona

minFeromona: mínimo nivel de feromona permitido para MAX-MIN Ant System

maxFeromona: máximo nivel de feromona permitido para MAX-MIN Ant System

#### Parámetros requeridos

Parámetros requeridos por AntSystem: alfa, beta, rho.

Parámetros requeridos por AntColonySystem: beta, rho, q0, phi

Parámetros requeridos por MaxMinAntSystem: alfa, beta, rho, minFeromona, maxFeromona

In [ ]:
alfa = 1
beta = 4
rho = 0.2
q0 = 0.7
phi = 0.2
minFeromona = 0
maxFeromona = 0.1

#### Pruebas Ant System

In [ ]:
# Cambiar por el nombre del problema deseado
nombreProblema = 'eil51'
matrizAdy = cargaCasoTsplib('ALL_tsp/' + nombreProblema + '.tsp')
# Alterar este range si se quiere ejecutar pruebas con varios números de individuos.
# Por defecto, se empieza por un número de individuos igual al número de ciudades,
# y se va aumentando de 5 en 5 hasta 5 veces esa cifra.
for numHormigas in range(len(matrizAdy), 5 * len(matrizAdy), 5):
    # 20 ejecuciones con cada nº de hormigas
    ejecuciones = []
    for i in range(20):
        entorno = Entorno(matrizAdy)
        soluciones, tiempo = AntSystem(entorno, numHormigas)
        ejecuciones.append((tiempo, soluciones))
    guardarResultados(ejecuciones, nombreProblema, 'AntSystem', numHormigas)

#### Pruebas Ant Colony System

In [ ]:
# Cambiar por el nombre del problema deseado
nombreProblema = 'eil51'
matrizAdy = cargaCasoTsplib('ALL_tsp/' + nombreProblema + '.tsp')
# Alterar este range si se quiere ejecutar pruebas con varios números de individuos.
# Por defecto, se empieza por un número de individuos igual al número de ciudades,
# y se va aumentando de 5 en 5 hasta 5 veces esa cifra.
for numHormigas in range(len(matrizAdy), 5 * len(matrizAdy), 5):
    # 20 ejecuciones con cada nº de hormigas
    ejecuciones = []
    for i in range(20):
        entorno = Entorno(matrizAdy)
        soluciones, tiempo = AntColonySystem(entorno, numHormigas)
        ejecuciones.append((tiempo, soluciones))
    guardarResultados(ejecuciones, nombreProblema, 'AntColonySystem', numHormigas)

#### Pruebas Max Min Ant System

In [ ]:
# Cambiar por el nombre del problema deseado
nombreProblema = 'eil51'
matrizAdy = cargaCasoTsplib('ALL_tsp/' + nombreProblema + '.tsp')
# Cambiar maxFeromona según el problema
# Sabemos que la mejor solución encontrada de eil51 es de longitud 426.
# Sabemos que la mejor solución encontrada de eil76 es de longitud 538.
# Sabemos que la mejor solución encontrada de st70 es de longitud 675.
# Sabemos que la mejor solución encontrada de berlin52 es de longitud 7544.
# Sabemos que la mejor solución encontrada de pr76 es de longitud 108159.
maxFeromona = 1/(rho * 426)
# Alterar este range si se quiere ejecutar pruebas con varios números de individuos.
# Por defecto, se empieza por un número de individuos igual al número de ciudades,
# y se va aumentando de 5 en 5 hasta 5 veces esa cifra.
for numHormigas in range(len(matrizAdy), 5 * len(matrizAdy), 5):
    # 20 ejecuciones con cada nº de hormigas
    ejecuciones = []
    for i in range(20):
        entorno = Entorno(matrizAdy)
        soluciones, tiempo = MaxMinAntSystemo(entorno, numHormigas)
        ejecuciones.append((tiempo, soluciones))
    guardarResultados(ejecuciones, 'eil51', 'MaxMinAntSystem', numHormigas)